<a href="https://colab.research.google.com/github/FLucasRodrigues/Pesquisa-Operacional/blob/main/Fila_no_supermercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

# Função para simular o comportamento dos clientes
def cliente(env, nome, caixas_autoatendimento, caixas_normais):
    print(f"{nome} chega ao supermercado em {env.now:.0f} minutos")

    # Escolher aleatoriamente o tipo de caixa
    tipo_caixa = random.choices(["autoatendimento", "normal"], weights=[0.3, 0.7])[0]

    # Entrar na fila do tipo de caixa escolhido
    if tipo_caixa == "autoatendimento":
        with caixas_autoatendimento.request() as req:
            yield req
            print(f"{nome} começa a ser atendido em um caixa de autoatendimento em {env.now:.0f} minutos")
            yield env.timeout(random.randint(1, 5))  # Tempo de atendimento entre 1 e 5 unidades de tempo
            print(f"{nome} termina de ser atendido em um caixa de autoatendimento em {env.now:.0f} minutos")
    else:
        with caixas_normais.request() as req:
            yield req
            print(f"{nome} começa a ser atendido em um caixa normal em {env.now:.0f} minutos")
            yield env.timeout(random.randint(1, 5))  # Tempo de atendimento entre 1 e 5 unidades de tempo
            print(f"{nome} termina de ser atendido em um caixa normal em {env.now:.0f} minutos")

# Função para gerar clientes de forma aleatória
def gerador_clientes(env, caixas_autoatendimento, caixas_normais):
    i = 0
    while True:
        i += 1
        yield env.timeout(random.expovariate(1/5))  # Intervalo de chegada dos clientes com média de 5 unidades de tempo
        env.process(cliente(env, f"Cliente {i}", caixas_autoatendimento, caixas_normais))

# Configuração da simulação
env = simpy.Environment()
caixas_autoatendimento = simpy.Resource(env, capacity=3)  # 3 caixas de autoatendimento
caixas_normais = simpy.Resource(env, capacity=4)  # 4 caixas normais
env.process(gerador_clientes(env, caixas_autoatendimento, caixas_normais))

# Execução da simulação
env.run(until=300)  # Rodar a simulação por 300 minutos

